In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from embedding_manager import *

In [69]:
data = pd.read_csv("data/transformed_attribute_data.csv")
data

,cod_modelo_color,cane_height_type,closure_placement,heel_shape_type,knit_structure,length_type,neck_lapel_type,silhouette_type,sleeve_length_type,toecap_type,waist_type,woven_structure
0,81_1034451,INVALID,INVALID,INVALID,INVALID,INVALID,Hawaiano/Bowling,Regular,Corta,INVALID,INVALID,INVALID
1,81_1034525,INVALID,INVALID,INVALID,INVALID,INVALID,Hawaiano/Bowling,Regular,Corta,INVALID,INVALID,INVALID
2,81_1035318,INVALID,INVALID,INVALID,INVALID,Largo,INVALID,INVALID,INVALID,INVALID,INVALID,INVALID
3,81_1035321,INVALID,INVALID,INVALID,INVALID,Largo,INVALID,INVALID,INVALID,INVALID,INVALID,INVALID
4,81_1035361,INVALID,INVALID,INVALID,INVALID,Largo,Capucha,INVALID,INVALID,INVALID,INVALID,INVALID
...,...,...,...,...,...,...,...,...,...,...,...,...
33438,86_9893076,INVALID,INVALID,INVALID,INVALID,Standard,Redondo,Slim,Tirante Fino,INVALID,INVALID,Ligero
33439,86_9893077,INVALID,INVALID,INVALID,INVALID,Standard,Redondo,Slim,Tirante Fino,INVALID,INVALID,Ligero
33440,86_9893868,INVALID,INVALID,INVALID,INVALID,Standard,Redondo,Slim,Tirante Fino,INVALID,INVALID,Ligero
33441,86_9893898,INVALID,Cierre Delantero,INVALID,INVALID,Standard,Pico,Recto,INVALID,INVALID,INVALID,Medio


In [70]:
ATTR_SLICE = "silhouette_type"

def get_df_processed(data, attr_slice, N_min=5, embedding_kind="long"):
    df = data[["cod_modelo_color", attr_slice]].copy()

    df = add_embeddings_to_df(df)
    df = add_attr_sim(df, embedding_kind=embedding_kind)
    df = add_subattr_sim(df, attr_slice, embedding_kind=embedding_kind)

    df = df.drop(columns=["cod_modelo_color"])

    # drop all attributes that appear less than N_min times
    drop_vals = df[attr_slice].value_counts()[df[attr_slice].value_counts() < N_min].index
    df = df[~df[attr_slice].isin(drop_vals)]

    return df


df = get_df_processed(data, ATTR_SLICE)

df = df[df[ATTR_SLICE] != "INVALID"].reset_index(drop=True)

In [71]:
df

,silhouette_type,embedding,silhouette_type_sim,Regular_sim,Slim_sim,5 Bolsillos_sim,Jogger_sim,Modern slim_sim,Chino_sim,Recto_sim,...,Halter_sim,Wide leg_sim,Paperbag_sim,Relaxed_sim,Tapered_sim,Bandeau_sim,Superslim_sim,Loose_sim,Carrot_sim,Parachute_sim
0,Regular,"[-0.17474917, 0.16217516, 0.40260923, -0.86747...",0.156534,0.177247,0.227847,0.195042,0.128015,0.232433,0.201189,0.223664,...,0.176627,0.157053,0.174292,0.225995,0.151041,0.167517,0.181443,0.210947,0.184354,0.211086
1,Regular,"[-0.24003744, 0.06450614, 0.8397347, -1.057063...",0.169158,0.189527,0.219528,0.189646,0.144634,0.222876,0.183011,0.218124,...,0.153930,0.147677,0.148163,0.234617,0.131399,0.127442,0.188263,0.205634,0.181059,0.214062
2,Regular,"[-0.18954061, 0.13276234, 1.1418515, -1.095528...",0.164281,0.184473,0.225257,0.163088,0.112088,0.219316,0.190101,0.222151,...,0.166113,0.153334,0.160998,0.205920,0.151870,0.159130,0.159468,0.189395,0.186586,0.206010
3,Regular,"[-0.4249166, -0.15395549, 0.89751333, -0.96329...",0.134716,0.159264,0.205074,0.172006,0.121808,0.182036,0.170056,0.197636,...,0.156975,0.139313,0.152826,0.196596,0.097377,0.127678,0.156017,0.166727,0.170418,0.187499
4,Slim,"[0.14905208, -0.6791551, 0.13594645, -0.010792...",0.213491,0.196758,0.285773,0.220547,0.149827,0.317527,0.251223,0.252150,...,0.154280,0.146752,0.135675,0.200696,0.180283,0.147151,0.238127,0.205110,0.188071,0.162682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52810,Slim,"[0.17486537, -0.17013556, 0.49440274, -0.78616...",0.223000,0.164816,0.228584,0.160026,0.139210,0.202625,0.157275,0.218612,...,0.227943,0.096741,0.164429,0.187346,0.138408,0.186359,0.201728,0.179231,0.168418,0.136206
52811,Recto,"[-0.06623721, 0.6834561, 0.6908933, -0.3276321...",0.223923,0.176295,0.226420,0.162233,0.174149,0.195463,0.200494,0.181695,...,0.255880,0.200840,0.240688,0.217808,0.130920,0.206714,0.171909,0.165166,0.190352,0.208629
52812,Recto,"[0.1940485, 0.1506823, 0.34528264, -0.7869217,...",0.271420,0.168618,0.242348,0.188218,0.132017,0.225735,0.187134,0.211903,...,0.305782,0.154742,0.217479,0.190759,0.174562,0.271185,0.212547,0.184450,0.204888,0.154048
52813,Recto,"[0.24107921, -0.58900714, 0.58061886, 0.381187...",0.227391,0.186564,0.218804,0.133767,0.147204,0.210627,0.178007,0.205229,...,0.149975,0.224720,0.192901,0.210970,0.123895,0.166795,0.191073,0.174084,0.171294,0.184070


In [73]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# define the features and target
X = df.drop(columns=[ATTR_SLICE, "embedding"])
y = df[ATTR_SLICE]


# encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# define the model
model = xgb.XGBClassifier(objective="multi:softmax", num_class=len(y.unique()))
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.580800908832718

In [ ]:
all_attrs = data[ATTR_SLICE].unique()

Index(['cod_modelo_color', 'cane_height_type', 'closure_placement',
       'heel_shape_type', 'knit_structure', 'length_type', 'neck_lapel_type',
       'silhouette_type', 'sleeve_length_type', 'toecap_type', 'waist_type',
       'woven_structure'],
      dtype='object')